In [1]:
import pandas as pd

payload = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
df = payload[0]

In [2]:
df.head()

,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M,reports,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1976-08-09,66740,1902
1,AOS,A. O. Smith,reports,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
3,ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ABMD,Abiomed,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981


In [3]:
df["Symbol"]

0       MMM
1       AOS
2       ABT
3      ABBV
4      ABMD
       ... 
500     YUM
501    ZBRA
502     ZBH
503    ZION
504     ZTS
Name: Symbol, Length: 505, dtype: object

In [4]:
payload = pd.read_csv('companiesmarketcap.com - Largest American companies by market capitalization.csv')
payload

,Rank,Name,Symbol,marketcap,price (USD),country
0,1,Apple,AAPL,2896125755392,174.31,United States
1,2,Microsoft,MSFT,2361437716480,309.42,United States
2,3,Alphabet (Google),GOOG,1894950502400,2814.00,United States
3,4,Amazon,AMZN,1664530513920,3271.20,United States
4,5,Tesla,TSLA,1137984405504,1084.59,United States
...,...,...,...,...,...,...
95,96,CME Group,CME,86196830208,239.82,United States
96,97,3M,MMM,85199355904,149.69,United States
97,98,Micron Technology,MU,85067915264,76.18,United States
98,99,Crown Castle,CCI,81705918464,189.04,United States


In [5]:
import tweepy as tw
import requests

access_token = "WT90c6EeFymch8alktIS6Czjo"
access_token_secret = "rapSxOMSspBkqyUSkQ7QsCqoBSl00eJn0lqM44MypligPrSLI3 "
bearer_token = "AAAAAAAAAAAAAAAAAAAAAPQdbAEAAAAAip4dX3hfODMZcwy%2F4D9NGsqQlQY%3DhlWENfRydVk69RbpqLE4vNXku5nzgoVeSfYMekfHfDm6pM0LtN "

client = tw.Client(bearer_token=bearer_token,
                        access_token=access_token, 
                        access_token_secret=access_token_secret, 
                        return_type = requests.Response,
                        wait_on_rate_limit=True)

In [6]:
query = "#AAPL -is:retweet -is:reply lang:en"
# -is:retweets

tweets = client.search_recent_tweets(query=query, max_results=100, start_time="2022-04-01T00:00:00Z")

In [7]:
# Save data as dictionary
tweets_dict = tweets.json() 

# Extract "data" value from dictionary
tweets_data = tweets_dict['data'] 

# Transform to pandas Dataframe
df = pd.json_normalize(tweets_data)
df

,id,text
0,1510319696922517507,AAPL hasn’t moved at all today #aapl #stocks
1,1510303556305309701,"AAPL,Apple,Nasdaq\nPeriod:1D,Daily\n177,69 is ..."
2,1510296946321379331,Tomorrow Sunday @ 8: 30PM EST #Youtube live!!!...
3,1510247504734007302,$AAPL one of the most mentioned on wallstreetb...
4,1510236738958020608,SHEESH This week's gains from analysts is NUTS...
...,...,...
68,1509713820847026205,"BBG: #AAPL, #FB, and #AMZN could soon be added..."
69,1509695576585158661,He means #TimCook #AAPL $AAPL $RVN $BTC $ETH $...
70,1509695184749137923,Huge news! #stocks #StocksInFocus #StocksToTra...
71,1509690739315281930,first time using twitter #AMC #GME #BBIG #MULN...


In [8]:
df["text"][0]

'AAPL hasn’t moved at all today #aapl #stocks'

In [9]:
def clean_text(text):
    text = text.str.lower() # lowercase
    #text = text.str.replace(r"\#","") # replaces hashtags
    text = text.str.replace(r"http\S+","URL")  # remove URL addresses
    text = text.str.replace(r"@","")
    text = text.str.replace(r"\n"," ")
    #text = text.str.replace(r"[^A-Za-z0-9()!?\'\`\"]", " ")
    text = text.str.replace(":", "")
    text = text.str.replace("\s{2,}", " ")
    return text

In [10]:
clean_text(df["text"])[0]

<ipython-input-9-674e6284c65f>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  text = text.str.replace(r"http\S+","URL")  # remove URL addresses
<ipython-input-9-674e6284c65f>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  text = text.str.replace(r"\n"," ")
<ipython-input-9-674e6284c65f>:9: FutureWarning: The default value of regex will change from True to False in a future version.
  text = text.str.replace("\s{2,}", " ")


'aapl hasn’t moved at all today #aapl #stocks'

In [11]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import matplotlib.pyplot as plt
from math import log, sqrt
import pandas as pd
import numpy as np
import re

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/andreaspalmgren/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [12]:
words = word_tokenize(clean_text(df["text"])[59])
words = [w for w in words if len(w) > 2]

sw = stopwords.words('english')
words = [word for word in words if word not in sw]

stemmer = PorterStemmer()
words = [stemmer.stem(word) for word in words]

words

<ipython-input-9-674e6284c65f>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  text = text.str.replace(r"http\S+","URL")  # remove URL addresses
<ipython-input-9-674e6284c65f>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  text = text.str.replace(r"\n"," ")
<ipython-input-9-674e6284c65f>:9: FutureWarning: The default value of regex will change from True to False in a future version.
  text = text.str.replace("\s{2,}", " ")


['aapl', 'aapl', 'chart', 'tradingview', 'url']

In [13]:
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

from official.nlp import optimization

In [29]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [33]:
def get_sentence_embeding(sentences):
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']

get_sentence_embeding(words)

<tf.Tensor: shape=(5, 768), dtype=float32, numpy=
array([[-0.8625343 , -0.16709733,  0.26343432, ...,  0.21032488,
        -0.5796226 ,  0.82706755],
       [-0.8625343 , -0.16709733,  0.26343432, ...,  0.21032488,
        -0.5796226 ,  0.82706755],
       [-0.837774  , -0.07798962,  0.43875256, ...,  0.3040768 ,
        -0.49907103,  0.8374878 ],
       [-0.7846132 , -0.2180038 ,  0.38767603, ...,  0.39983296,
        -0.54361093,  0.82489705],
       [-0.8504085 , -0.37325853, -0.4750819 , ..., -0.2725301 ,
        -0.69858265,  0.87868434]], dtype=float32)>

In [34]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [35]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer_3 (KerasLayer)     {'input_type_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128)}                                                  

In [36]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)

In [37]:
model.fit(X_train, y_train, epochs=10)

NameError: name 'X_train' is not defined

In [ ]:
import yfinance as yf  
import matplotlib.pyplot as plt

In [ ]:
data = yf.download(['AAPL'],'2021-01-01','2022-03-30')
data

https://www.diva-portal.org/smash/get/diva2:1113099/FULLTEXT01.pdf